In [2]:
import sys
import os
import json

from pprint import pprint


nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

    
project_dir = os.path.join(os.getcwd(), '..')
print(sys.version)
print(os.getcwd())

pprint(sys.path)


3.6.5 (default, May  3 2018, 10:08:28) 
[GCC 5.4.0 20160609]
/home/viktor/dev/recept-scanning/notebooks
['',
 '/home/viktor/venvs/receipt/lib/python36.zip',
 '/home/viktor/venvs/receipt/lib/python3.6',
 '/home/viktor/venvs/receipt/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '/home/viktor/venvs/receipt/lib/python3.6/site-packages',
 '/home/viktor/venvs/receipt/lib/python3.6/site-packages/IPython/extensions',
 '/home/viktor/.ipython',
 '/home/viktor/dev/recept-scanning']


In [13]:
import boto3

boto_session = boto3.Session(profile_name='serverless-admin', region_name='eu-central-1')


ddb = boto_session.client('dynamodb')

In [39]:
c = ddb.put_item(
    TableName='gcv_responses',
    Item={
        'image_hash': dict(S='rumpe'),
        'gcv_response': dict(M=dict(textAnnotations=dict(L=[dict(S='anus')])))
    }
)

c

{'ResponseMetadata': {'RequestId': 'OLDEVUVVD17AH6S7N83E4QRT7NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 13 Sep 2018 19:05:57 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'OLDEVUVVD17AH6S7N83E4QRT7NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [43]:
tables = ddb.list_tables()
tables

scan = ddb.scan(TableName=tables['TableNames'][0])

scan.keys()

dict_keys(['Items', 'Count', 'ScannedCount', 'ResponseMetadata'])

In [3]:
from botocore.session import Session
from pynamodb.connection.base import Connection
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, MapAttribute, ListAttribute

from recread import config


class GcvResponseModel(Model):
    class Meta:
        table_name = 'gcv_responses'
        region = config.AWS_REGION_DEFAULT

    image_hash = UnicodeAttribute(hash_key=True)
    gcv_response = MapAttribute(default=None)


class ReceiptLinesModel(Model):
    class Meta:
        table_name = 'receipt_lines'
        region = config.AWS_REGION_DEFAULT

    image_hash = UnicodeAttribute(hash_key=True)
    receipt_lines = ListAttribute(default=None)
    receipt_lines_tokens = ListAttribute(default=None)


def monkeypatch_connection(profile=config.AWS_PROFILE_DEFAULT):
    @property
    def session(self):
        if getattr(self._local, 'session', None) is None:
            self._local.session = Session(profile=profile)
        return self._local.session

    Connection.session = session



monkeypatch_connection(profile='serverless-admin')

with open(os.path.join(project_dir, 'data', 'receipt_responses', 'kvittering-keiser.jpg.json'), 'r') as json_file:
    gcv_response = json.load(json_file)

item = GcvResponseModel.get('4ff7dd961835b8c8b33db0810d26a8532d5e7809e1071bf2d301d86ad8f79809')


item


In [8]:
with open(os.path.join(project_dir, 'data', 'receipt_responses', 'kvittering-ikea.jpg.json'), 'w') as json_output_file:
    json.dump(item.gcv_response.as_dict(), json_output_file)

In [5]:
import boto3
import pynamodb

from recread.dynamodb.pynamo.models import GcvResponseModel, ReceiptLinesModel


def save_gcv_response(gcv_response, image_hash):
    db_object = GcvResponseModel(image_hash)
    return db_object.update(actions=[
        GcvResponseModel.gcv_response.set(gcv_response),
    ])


def save_receipt_lines(receipt_lines, receipt_lines_tokens, image_hash):
    db_object = ReceiptLinesModel(image_hash)
    return db_object.update(actions=[
        ReceiptLinesModel.receipt_lines.set(receipt_lines),
        ReceiptLinesModel.receipt_lines_tokens.set(receipt_lines_tokens),
    ])


In [25]:
ReceiptLinesModel.create_table(write_capacity_units=5, read_capacity_units=5)

anus_item = save_receipt_lines(dict(rumpe='tiss'), [['anus', 'promp', 'tiss']], '123abc')

TypeError: 'NoneType' object is not iterable

In [11]:
penis_item = GcvResponseModel.get('123abc')
penis_item.__dict__

{'attribute_values': {'gcv_response': <pynamodb.attributes.MapAttribute at 0x7f1a5de72e10>,
  'image_hash': '123abc'}}

In [20]:
penis_item.image_hash

'123abc'

In [91]:
import hashlib

m = hashlib.blake2s()
m.update(b'')
type(m.hexdigest())

str

In [164]:
class Dog(object):
    def __init__(self, name='dog', color='black', parents=[], *args, **kwargs):
        self.name = name
        self.color = color
        self.parents = parents
        
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.name
    
    def __iter__(self):
        for key, value in self.__dict__.items():
            yield key, value
    
    
        

dogman = Dog('man', 'yellow', ['Dick'])

dict(dogman)

{'name': 'man', 'color': 'yellow', 'parents': ['Dick']}

In [26]:
env = {'LC_PAPER': 'nb_NO.UTF-8', 'XDG_VTNR': '7', '__GIT_PROMPT_IGNORE_STASH': '', 'LC_ADDRESS': 'nb_NO.UTF-8', 'SSH_AGENT_PID': '1808', 'XDG_SESSION_ID': 'c2', 'LC_MONETARY': 'nb_NO.UTF-8', 'XDG_GREETER_DATA_DIR': '/var/lib/lightdm-data/viktor', 'QT_STYLE_OVERRIDE': 'gtk', 'NVM_CD_FLAGS': '', 'GPG_AGENT_INFO': '/home/viktor/.gnupg/S.gpg-agent:0:1', 'TERM': 'xterm-256color', 'SHELL': '/bin/bash', 'VTE_VERSION': '4205', 'QT_LINUX_ACCESSIBILITY_ALWAYS_ON': '1', 'WINDOWID': '113246214', 'LC_NUMERIC': 'nb_NO.UTF-8', 'GTK_MODULES': 'gail:atk-bridge', '__GIT_PROMPT_SHOW_UPSTREAM': '', '__GIT_PROMPT_SHOW_UNTRACKED_FILES': 'all', 'NVM_DIR': '/home/viktor/.nvm', 'USER': 'viktor', 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=01;31:*.tzo=01;31:*.t7z=01;31:*.zip=01;31:*.z=01;31:*.Z=01;31:*.dz=01;31:*.gz=01;31:*.lrz=01;31:*.lz=01;31:*.lzo=01;31:*.xz=01;31:*.bz2=01;31:*.bz=01;31:*.tbz=01;31:*.tbz2=01;31:*.tz=01;31:*.deb=01;31:*.rpm=01;31:*.jar=01;31:*.war=01;31:*.ear=01;31:*.sar=01;31:*.rar=01;31:*.alz=01;31:*.ace=01;31:*.zoo=01;31:*.cpio=01;31:*.7z=01;31:*.rz=01;31:*.cab=01;31:*.jpg=01;35:*.jpeg=01;35:*.gif=01;35:*.bmp=01;35:*.pbm=01;35:*.pgm=01;35:*.ppm=01;35:*.tga=01;35:*.xbm=01;35:*.xpm=01;35:*.tif=01;35:*.tiff=01;35:*.png=01;35:*.svg=01;35:*.svgz=01;35:*.mng=01;35:*.pcx=01;35:*.mov=01;35:*.mpg=01;35:*.mpeg=01;35:*.m2v=01;35:*.mkv=01;35:*.webm=01;35:*.ogm=01;35:*.mp4=01;35:*.m4v=01;35:*.mp4v=01;35:*.vob=01;35:*.qt=01;35:*.nuv=01;35:*.wmv=01;35:*.asf=01;35:*.rm=01;35:*.rmvb=01;35:*.flc=01;35:*.avi=01;35:*.fli=01;35:*.flv=01;35:*.gl=01;35:*.dl=01;35:*.xcf=01;35:*.xwd=01;35:*.yuv=01;35:*.cgm=01;35:*.emf=01;35:*.ogv=01;35:*.ogx=01;35:*.aac=00;36:*.au=00;36:*.flac=00;36:*.m4a=00;36:*.mid=00;36:*.midi=00;36:*.mka=00;36:*.mp3=00;36:*.mpc=00;36:*.ogg=00;36:*.ra=00;36:*.wav=00;36:*.oga=00;36:*.opus=00;36:*.spx=00;36:*.xspf=00;36:', 'QT_ACCESSIBILITY': '1', 'DESKTOP_AUTOSTART_ID': '10ecf6ac954a813bd8153700565760719600000016020026', 'LC_TELEPHONE': 'nb_NO.UTF-8', 'XDG_SESSION_PATH': '/org/freedesktop/DisplayManager/Session0', 'XDG_SEAT_PATH': '/org/freedesktop/DisplayManager/Seat0', 'SSH_AUTH_SOCK': '/run/user/1000/keyring/ssh', 'SESSION_MANAGER': 'local/viktor-ThinkPad-T460s:@/tmp/.ICE-unix/1602,unix/viktor-ThinkPad-T460s:/tmp/.ICE-unix/1602', 'DEFAULTS_PATH': '/usr/share/gconf/cinnamon.default.path', 'VIRTUAL_ENV': '/home/viktor/venvs/receipt', 'XDG_CONFIG_DIRS': '/etc/xdg/xdg-cinnamon:/etc/xdg', 'PATH': '/home/viktor/venvs/receipt/bin:/home/viktor/venvs/receipt/bin:/home/viktor/.nvm/versions/node/v10.7.0/bin:/home/viktor/bin:/home/viktor/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/usr/local/go/bin:/home/viktor/go/bin:/snap/hugo/current/bin:/home/viktor/.local/bin', 'DESKTOP_SESSION': 'cinnamon', 'QT_QPA_PLATFORMTHEME': 'qgnomeplatform', 'LC_IDENTIFICATION': 'nb_NO.UTF-8', 'XDG_SESSION_TYPE': 'x11', 'PWD': '/home/viktor/dev/recept-scanning', 'LANG': 'en_US.UTF-8', 'GDM_LANG': 'en_US', 'MANDATORY_PATH': '/usr/share/gconf/cinnamon.mandatory.path', 'LC_MEASUREMENT': 'nb_NO.UTF-8', 'PS1': '(\\[\\033[0;34m\\]receipt\\[\\033[0;0m\\]) \\[\\033[0;32m\\]✔ \\[\\033[0;0m\\] \\[\\033[0;33m\\]\\w\\[\\033[0;0m\\] [\\[\\033[0;35m\\]${GIT_BRANCH}\\[\\033[0;0m\\]|\\[\\033[0;34m\\]✚ 2\\[\\033[0;0m\\]\\[\\033[0;36m\\]…21\\[\\033[0;0m\\]\\[\\033[0;0m\\]] \\n\\[\\033[1;34m\\]$(date +%H:%M)\\[\\033[0;0m\\] $ ', 'GDMSESSION': 'cinnamon', 'XDG_SEAT': 'seat0', 'HOME': '/home/viktor', 'SHLVL': '1', 'LANGUAGE': 'en_US', 'GIT_BRANCH': 'master', '__GIT_PROMPT_SHOW_CHANGED_FILES_COUNT': '1', 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated', 'GTK_OVERLAY_SCROLLING': '1', '__GIT_PROMPT_IGNORE_SUBMODULES': '', 'XDG_SESSION_DESKTOP': 'cinnamon', 'LOGNAME': 'viktor', 'XDG_DATA_DIRS': '/usr/share/cinnamon:/usr/share/gnome:/home/viktor/.local/share/flatpak/exports/share:/var/lib/flatpak/exports/share:/usr/local/share:/usr/share:/var/lib/snapd/desktop', 'DBUS_SESSION_BUS_ADDRESS': 'unix:abstract=/tmp/dbus-FG1pQQFcvB,guid=fa7c26a4f72ba1bd819f29345b9cd859', 'NVM_BIN': '/home/viktor/.nvm/versions/node/v10.7.0/bin', 'LESSOPEN': '| /usr/bin/lesspipe %s', 'XDG_RUNTIME_DIR': '/run/user/1000', 'DISPLAY': ':0', 'XDG_CURRENT_DESKTOP': 'X-Cinnamon', 'LESSCLOSE': '/usr/bin/lesspipe %s %s', 'XAUTHORITY': '/home/viktor/.Xauthority', 'LC_NAME': 'nb_NO.UTF-8', '_': '/home/viktor/.nvm/versions/node/v10.7.0/bin/sls', 'IS_LOCAL': 'true', 'LD_LIBRARY_PATH': '/usr/local/lib64/node-v4.3.x/lib:/lib64:/usr/lib64:/var/runtime:/var/runtime/lib:/var/task:/var/task/lib', 'LAMBDA_TASK_ROOT': '/var/task', 'LAMBDA_RUNTIME_DIR': '/var/runtime', 'AWS_REGION': 'us-east-1', 'AWS_DEFAULT_REGION': 'us-east-1', 'AWS_LAMBDA_LOG_GROUP_NAME': '/aws/lambda/serverless-receipt-scanning-dev-hello', 'AWS_LAMBDA_LOG_STREAM_NAME': '2016/12/02/[$LATEST]f77ff5e4026c45bda9a9ebcec6bc9cad', 'AWS_LAMBDA_FUNCTION_NAME': 'serverless-receipt-scanning-dev-hello', 'AWS_LAMBDA_FUNCTION_MEMORY_SIZE': '1024', 'AWS_LAMBDA_FUNCTION_VERSION': '$LATEST', 'NODE_PATH': '/var/runtime:/var/task:/var/runtime/node_modules'}
env

{'LC_PAPER': 'nb_NO.UTF-8',
 'XDG_VTNR': '7',
 '__GIT_PROMPT_IGNORE_STASH': '',
 'LC_ADDRESS': 'nb_NO.UTF-8',
 'SSH_AGENT_PID': '1808',
 'XDG_SESSION_ID': 'c2',
 'LC_MONETARY': 'nb_NO.UTF-8',
 'XDG_GREETER_DATA_DIR': '/var/lib/lightdm-data/viktor',
 'QT_STYLE_OVERRIDE': 'gtk',
 'NVM_CD_FLAGS': '',
 'GPG_AGENT_INFO': '/home/viktor/.gnupg/S.gpg-agent:0:1',
 'TERM': 'xterm-256color',
 'SHELL': '/bin/bash',
 'VTE_VERSION': '4205',
 'QT_LINUX_ACCESSIBILITY_ALWAYS_ON': '1',
 'WINDOWID': '113246214',
 'LC_NUMERIC': 'nb_NO.UTF-8',
 'GTK_MODULES': 'gail:atk-bridge',
 '__GIT_PROMPT_SHOW_UPSTREAM': '',
 '__GIT_PROMPT_SHOW_UNTRACKED_FILES': 'all',
 'NVM_DIR': '/home/viktor/.nvm',
 'USER': 'viktor',
 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.lzma=01;31:*.tlz=01;31:*.txz=0